# todo

todo

In [ ]:
# Imports & config
# Ensure required libraries are installed
!pip install python-dotenv openai pypdf2 gradio

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_BASE_URL = os.getenv('OPENAI_BASE_URL', 'https://api.openai.com/v1')
OPENAI_MODEL = os.getenv('OPENAI_MODEL', 'gpt-4o-mini')

HEADERS = {
    'Authorization': f'Bearer {OPENAI_API_KEY}' if OPENAI_API_KEY else '',
    'Content-Type': 'application/json',
}

DRY_RUN = True


## Prompts and Resources loading
Load from disk a CV in pdf format and a presentation letter as text file.

In [ ]:
reader = PdfReader("assets/me/CV.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

print(linkedin)

with open("assets/me/about.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Matteo Rigoni"

In [ ]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

system_prompt

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Minimal OpenAI‑compatible client + helpers
- `chat(messages)` calls the Chat Completions API or returns **mocked** JSON when `DRY_RUN=True`.
- `jparse(s)` safely parses JSON (with a small fallback).

In [3]:
def jparse(s: str):
    try:
        return json.loads(s)
    except json.JSONDecodeError:
        s2 = s.strip().strip('`\n ')
        first = s2.find('{'); last = s2.rfind('}')
        if first != -1 and last != -1:
            return json.loads(s2[first:last+1])
        raise

def _mock_response_for(prompt: str) -> str:
    # Heuristic mock based on which prompt it is
    if 'business area' in prompt and 'Return JSON with' in prompt and 'business_area' in prompt:
        return json.dumps({
            'business_area': 'Property Management (SMB)',
            'why_promising': 'Fragmented operations, thin margins, and repetitive workflows enable agents to coordinate vendors and tenants end‑to‑end.'
        })
    if 'identify one high‑value pain‑point' in prompt:
        return json.dumps({
            'pain_point': 'Coordinating maintenance across units: triage, scheduling, and follow‑ups',
            'who_is_affected': 'Property managers and tenants in buildings with 50–500 units',
            'current_workarounds': 'Shared inboxes, spreadsheets, and phone tag with vendors',
            'why_hard': 'Unstructured requests, multi‑party constraints in scheduling, poor status visibility.'
        })
    if 'Propose an Agentic AI solution' in prompt:
        return json.dumps({
            'solution_name': 'FixFlow Agent Mesh',
            'agent_roles': ['Triage Agent','Planner','Vendor Liaison','QA/Evaluator'],
            'data_sources': ['Tenant portal messages','Calendar APIs','Vendor CRM','Photos/Videos'],
            'key_actions': ['Parse issue','Quote ETA','Auto‑schedule vendor','Confirm completion'],
            'success_metrics': ['Time‑to‑schedule','First‑visit fix %','CSAT'],
            'risks': ['Hallucinated scheduling','Vendor no‑shows','Edge cases'],
            'quick_prototype': [
                'Webhook intake from tenant form',
                'LLM triage → category+priority',
                'Heuristic + calendar API scheduling',
                'Email/SMS notifications',
                'Simple dashboard for overrides',
                'Audit log and trace capture'
            ]
        })
    # default fallthrough
    return json.dumps({'note': 'mock'})

def chat(messages):
    if DRY_RUN:
        # Return a plausible mock based on the last user message
        last_user = next((m['content'] for m in reversed(messages) if m['role']=='user'), '')
        return _mock_response_for(last_user)
    url = f"{OPENAI_BASE_URL}/chat/completions"
    resp = requests.post(
        url,
        headers=HEADERS,
        json={
            'model': OPENAI_MODEL,
            'messages': messages,
            'temperature': 0.7,
        },
        timeout=60,
    )
    resp.raise_for_status()
    data = resp.json()
    return data['choices'][0]['message']['content'].strip()

## Run the three‑call flow
This cell performs the full chain and writes `trace.json`.

In [4]:
def run_chain():
    trace = {'calls': [], 'created_at': datetime.utcnow().isoformat() + 'Z'}
    messages = [SYSTEM]

    # 1) Business area
    messages.append({'role': 'user', 'content': PROMPTS['area']})
    raw1 = chat(messages)
    out1 = jparse(raw1)
    trace['calls'].append({'step': 1, 'prompt': PROMPTS['area'], 'response': out1})

    # 2) Pain‑point
    p2 = PROMPTS['pain'].replace('{{business_area}}', out1['business_area'])
    messages.append({'role': 'user', 'content': p2})
    raw2 = chat(messages)
    out2 = jparse(raw2)
    trace['calls'].append({'step': 2, 'prompt': p2, 'response': out2})

    # 3) Agentic solution
    p3 = PROMPTS['solution'] \
        .replace('{{business_area}}', out1['business_area']) \
        .replace('{{pain_point}}', out2['pain_point'])
    messages.append({'role': 'user', 'content': p3})
    raw3 = chat(messages)
    out3 = jparse(raw3)
    trace['calls'].append({'step': 3, 'prompt': p3, 'response': out3})

    # Save trace
    with open('trace.json', 'w') as f:
        json.dump(trace, f, indent=2)

    return out1, out2, out3

area, pain, solution = run_chain()
print('=== Result ===')
print('Business Area:', area.get('business_area'))
print('Pain‑Point   :', pain.get('pain_point'))
print('Solution     :', solution.get('solution_name'))

=== Result ===
Business Area: Property Management (SMB)
Pain‑Point   : Coordinating maintenance across units: triage, scheduling, and follow‑ups
Solution     : FixFlow Agent Mesh


## Inspect the trace.json

In [5]:
with open('trace.json') as f:
    trace = json.load(f)
print(json.dumps(trace, indent=2))

{
  "calls": [
    {
      "step": 1,
      "prompt": "Task: Pick one business area that\u2019s promising for an Agentic AI opportunity.\nReturn JSON with:\n- business_area: string\n- why_promising: string (\u2264 2 sentences)\nConstraints: Keep it under 80 words total.",
      "response": {
        "business_area": "Property Management (SMB)",
        "why_promising": "Fragmented operations, thin margins, and repetitive workflows enable agents to coordinate vendors and tenants end\u2011to\u2011end."
      }
    },
    {
      "step": 2,
      "prompt": "Task: In the business area \"Property Management (SMB)\", identify one high\u2011value pain\u2011point.\nReturn JSON with:\n- pain_point: string\n- who_is_affected: string\n- current_workarounds: string\n- why_hard: string\nConstraints: Keep it under 120 words total.",
      "response": {
        "pain_point": "Coordinating maintenance across units: triage, scheduling, and follow\u2011ups",
        "who_is_affected": "Property managers